In [9]:
using Flux
using ONNX
using ONNXNaiveNASflux
using Flux, Statistics
using Flux: onehotbatch, onecold, logitcrossentropy, @epochs#, @treelike
using MLDatasets
using ResNet

In [31]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    Conv((3, 3), 8 => 8, relu, pad=SamePad(), stride=(2, 2)),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(128, 100, relu),
    Dense(100, 10)
])
input_shape = (32, 32, 3, 5)
ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/mlp_small.onnx", model, input_shape)

67023

In [26]:
model = Chain([
    Conv((3, 3), 3 => 8, relu, pad=SamePad(), stride=(2, 2)), #pad=SamePad() ensures size(output,d) == size(x,d) / stride.
    BatchNorm(8),
    MeanPool((2,2)),
    SkipConnection(
        Chain([
            Conv((5, 5), 8 => 8, relu, pad=SamePad(), stride=(1, 1))
            ]),
        +
    ),
    #ConvTranspose((3, 3), 8 => 4, relu, pad=SamePad(), stride=(2, 2)),#pad=SamePad() ensures size(output,d) == size(x,d) * stride.
    Flux.flatten,
    Dense(512, 100, relu),
    Dense(100, 10)
])
input_shape = (32, 32, 3, 5)
ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/mlp.onnx", model, input_shape)

218065

In [23]:
input = randn(32, 32, 3, 5)
println(size(model(input)))

(10, 5)


In [24]:
input_shape = (32, 32, 3, 5)
ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/mlp.onnx", model, input_shape)

67017

In [27]:
function build_resnet18()
    return Chain(
        Conv((7, 7), 3 => 64, stride = 2, pad = 3),
        BatchNorm(64),
        MeanPool((3, 3), stride = 2, pad = 1),
        ResidualBlock(64, 64),
        ResidualBlock(64, 64),
        MeanPool((3, 3), stride = 2, pad = 1),
        ResidualBlock(64, 128),
        ResidualBlock(128, 128),
        MeanPool((3, 3), stride = 2, pad = 1),
        ResidualBlock(128, 256),
        ResidualBlock(256, 256),
        MeanPool((3, 3), stride = 2, pad = 1),
        ResidualBlock(256, 512),
        ResidualBlock(512, 512),
        MeanPool((7, 7)),
        Flux.flatten,
        Dense(512, 10),
        softmax
    )
end

function ResidualBlock(in_channels, out_channels)
return Chain(
    Conv((3, 3), in_channels => out_channels, stride = 1, pad = 1),
    BatchNorm(out_channels),
    x -> relu.(x),
    Conv((3, 3), out_channels => out_channels, stride = 1, pad = 1),
    BatchNorm(out_channels),
    x -> x + identity(x),
    x -> relu.(x)
)
end
model = build_resnet18()
input_shape = (32, 32, 3, 1)
ONNXNaiveNASflux.save("/home/verification/ModelVerification.jl/resnet.onnx", model, input_shape)

44087982